In [1]:
import pandas as pd

In [2]:
df_raw = pd.read_csv('../data/full_data.csv')
df = df_raw

### Finetune versions

In [3]:
# Create ongoing version_id that is unique for each version page_id combination
df['version_id_ong'] = df['page_id'].astype(str) + '_' + df['version_id'].astype(str)
df[['version_id_ong', 'page_id', 'version_id']].head()

,version_id_ong,page_id,version_id
0,1037_0,1037,0
1,1037_0,1037,0
2,1037_0,1037,0
3,1037_0,1037,0
4,1037_0,1037,0


In [4]:
pd.set_option('display.max_colwidth', None)

# Columns that differ on a daily basis and need to be aggregated with a certain rule
col_agg = ['daily_likes', 'daily_dislikes', 'video_play', 'page_impressions', 'clickouts', 'external_clicks', 'external_impressions']

# Columns that don't need to be aggregated but are the same for each version
all_columns = df.columns.tolist()
col = [c for c in all_columns if c not in col_agg]

# Group by version
df_agg = df[col].groupby('version_id_ong').first()
df_agg.head()

,old_index,page_id,date,url,version_id,publish_date,word_count,words_scraped,classification_product,classification_type,page_name,authors,author_scraped,title,h1,abstract,last_update,image_url
version_id_ong,,,,,,,,,,,,,,,,,,
1010012_0,26796,1010012,2023-05-20,https://efahrer.chip.de/solaranlagen/hersteller-reduziert-fast-alle-solargeneratoren-diese-koennen-wir-empfehlen_1010012,0,2018-01-01,0.0,1844,Solaranlagen,Deal,efa-1010012 | Hersteller reduziert fast alle Solargeneratoren: Diese können wir empfehlen,Sebastian Barsch,Sebastian Barsch,Hersteller reduziert fast alle Solargeneratoren: Diese können wir empfehlen,Hersteller reduziert fast alle Solargeneratoren: Diese können wir empfehlen,"Bluetti ist einer der großen Hersteller von Solargeneratoren. Jetzt zu Halloween rabattiert er ein Großteil seines Sortiments, doch nicht alles lohnt sich. Wir haben daher die drei besten Angebote zusammengefasst.",2022-10-27,https://im-efahrer.chip.de/files/bluetti-aufmacher-635aab8d6d430.jpg?imPolicy=IfOrientation&width=1200&height=630&color=%23000000&hash=b7e6e70ca0023fdfa0066f715197e5326a6e4ba8032181800713f9bef85a7890
1010022_0,129383,1010022,2023-04-29,https://efahrer.chip.de/news/mythos-solaranlage-warum-sie-bei-einem-blackout-keinen-strom-liefert_1010022,0,2023-04-29,570.0,1268,Solaranlagen,News,efa-1010022 | Beliebter Irrglaube: Warum PV-Anlagen beim Blackout auch keinen Strom liefern,Aslan Berse,Aslan Berse,Beliebter Irrglaube: Warum PV-Anlagen beim Blackout auch keinen Strom liefern,Trotz Blackout den eigene Solarstrom nutzen: Warum das nicht so einfach geht,"Ein bundesweiter Blackout könnte unser techniklastiges Leben schon ziemlich einschränken. Mit einer Solaranlage würde man dieses Problem umgehen können, so der Glaube vieler Hausbesitzer. Ein nachvollziehbarer Gedanke, schließlich wird durch die eigene PV-Anlage Solarstrom durch die Sonneneinstrahlung generiert. Dabei sind noch viele weitere Faktoren unabdingbar, um bei einem Stromausfall weiterhin den eigenen Solarstrom nutzen zu können. Welche diese sind, erfahren Sie durch EFAHRER.com",2023-06-10,https://im-efahrer.chip.de/files/black-pv-635bd11121f60.jpg?imPolicy=IfOrientation&width=1200&height=630&color=%23000000&hash=0dfc2924bc65d476ae9dfd7fdfe6896671f545dcb9184a70631e2ecebe154f6b
1010022_1,20689,1010022,2023-06-10,https://efahrer.chip.de/news/mythos-solaranlage-warum-sie-bei-einem-blackout-keinen-strom-liefert_1010022,1,2023-06-10,594.0,1268,Solaranlagen,News,efa-1010022 | Beliebter Irrglaube: Warum PV-Anlagen beim Blackout auch keinen Strom liefern,Aslan Berse,Aslan Berse,Beliebter Irrglaube: Warum PV-Anlagen beim Blackout auch keinen Strom liefern,Trotz Blackout den eigene Solarstrom nutzen: Warum das nicht so einfach geht,"Ein bundesweiter Blackout könnte unser techniklastiges Leben schon ziemlich einschränken. Mit einer Solaranlage würde man dieses Problem umgehen können, so der Glaube vieler Hausbesitzer. Ein nachvollziehbarer Gedanke, schließlich wird durch die eigene PV-Anlage Solarstrom durch die Sonneneinstrahlung generiert. Dabei sind noch viele weitere Faktoren unabdingbar, um bei einem Stromausfall weiterhin den eigenen Solarstrom nutzen zu können. Welche diese sind, erfahren Sie durch EFAHRER.com",2023-06-10,https://im-efahrer.chip.de/files/black-pv-635bd11121f60.jpg?imPolicy=IfOrientation&width=1200&height=630&color=%23000000&hash=0dfc2924bc65d476ae9dfd7fdfe6896671f545dcb9184a70631e2ecebe154f6b
1010045_0,106286,1010045,2023-02-23,https://efahrer.chip.de/e-bikes/versteckt-auf-jedem-rad-diese-zahl-beim-e-bike-ist-lebenswichtig_1010045,0,2023-02-23,430.0,1066,E-Bike,News,efa-1010045 | Kleiner Aufkleber auf dem Rad: Diese Zahl ist beim E-Bike lebenswichtig,Vanessa Finkler,Vanessa Finkler,Kleiner Aufkleber auf dem Rad: Diese Zahl ist beim E-Bike lebenswichtig,Rahmenbruchgefahr: Ignorieren Sie niemals diesen Aufkleber am E-Bike,"Der jüngste Fall bei den Babboe-Lastenrädern zeigt es: Rahmenbrüche kommen auch bei E-Bikes vor. Deshalb sollten Sie wissen, was Ihr Ra

<span style='color:red'>Can be ignored

Example:

In [5]:
pd.set_option('display.max_colwidth', 20)
df_agg[df_agg['page_id'] == 1010045]

,old_index,page_id,date,url,version_id,publish_date,word_count,words_scraped,classification_product,classification_type,page_name,authors,author_scraped,title,h1,abstract,last_update,image_url
version_id_ong,,,,,,,,,,,,,,,,,,
1010045_0,106286,1010045,2023-02-23,https://efahrer....,0,2023-02-23,430.0,1066,E-Bike,News,efa-1010045 | Kl...,Vanessa Finkler,Vanessa Finkler,Kleiner Aufkleb...,Rahmenbruchgefah...,Der jüngste Fall...,2024-02-25,https://im-efahr...
1010045_1,84721,1010045,2023-06-22,https://efahrer....,1,2023-06-22,431.0,1066,E-Bike,News,efa-1010045 | Kl...,Vanessa Finkler,Vanessa Finkler,Kleiner Aufkleb...,Rahmenbruchgefah...,Der jüngste Fall...,2024-02-25,https://im-efahr...
1010045_2,105160,1010045,2023-08-12,https://efahrer....,2,2023-08-12,440.0,1066,E-Bike,News,efa-1010045 | Kl...,Vanessa Finkler,Vanessa Finkler,Kleiner Aufkleb...,Rahmenbruchgefah...,Der jüngste Fall...,2024-02-25,https://im-efahr...
1010045_3,100948,1010045,2023-10-15,https://efahrer....,3,2023-10-15,397.0,1066,E-Bike,News,efa-1010045 | Kl...,Vanessa Finkler,Vanessa Finkler,Kleiner Aufkleb...,Rahmenbruchgefah...,Der jüngste Fall...,2024-02-25,https://im-efahr...
1010045_4,25668,1010045,2023-12-23,https://efahrer....,4,2023-11-19,395.0,1066,E-Bike,News,efa-1010045 | Kl...,Vanessa Finkler,Vanessa Finkler,Kleiner Aufkleb...,Rahmenbruchgefah...,Der jüngste Fall...,2024-02-25,https://im-efahr...
1010045_5,95631,1010045,2023-12-25,https://efahrer....,5,2023-12-24,397.0,1066,E-Bike,News,efa-1010045 | Kl...,Vanessa Finkler,Vanessa Finkler,Kleiner Aufkleb...,Rahmenbruchgefah...,Der jüngste Fall...,2024-02-25,https://im-efahr...
1010045_6,48648,1010045,2024-02-26,https://efahrer....,6,2024-02-25,400.0,1066,E-Bike,News,efa-1010045 | Kl...,Vanessa Finkler,Vanessa Finkler,Kleiner Aufkleb...,Rahmenbruchgefah...,Der jüngste Fall...,2024-02-25,https://im-efahr...


### Target Variables:
Impressions, Clicks, CTR (click-through-rate). The latter we create in the next step:

In [ ]:
# Calculate the Click through rate based on external clicks and impressions
df['ctr'] = df['external_clicks'] / df['external_impressions'] *100

### Features:
Category, 
Image, 
H1, 
Abstract, 
URL, 
Title, 
Word count

Extract the last part of the URL to analyze it and inhibit duplicate data with classification_type

In [ ]:
# Function to extract last part of URL
def extract_last_part(url):
    return url.rsplit('/', 1)[-1]

# Apply the function to create a new column
df.loc[:, 'url_text'] = df['url'].apply(extract_last_part)

Add all keywords from URLs and group them in a list per version

In [ ]:
# # URL keywords into list 
# tbd

# # drop duplicates
# tbd

In [ ]:
# Title length
df['title_len'] = df['title'].str.len()

# H1 length
df['h1_len'] = df['h1'].str.len()

# Abstract length
df['abstract_len'] = df['abstract'].str.len()

In [ ]:
df['url_len'] = df['url_text'].str.len()

In [11]:
#df['url_keywords'] = 

In [13]:
## Does the page contain a video or image
#df['media_type'] = 

In [14]:
# One hot encode category & type


Extract the most important words from title, h1, abstract, url with NLP (e.g. TF-IDF )

In [15]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

[nltk_data] Downloading package wordnet to /Users/clara/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/clara/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [16]:
col = ['title', 'h1', 'abstract', 'url_text']

In [19]:
def vectorize_text(column, df=df):
    col = df[column]
    vect = CountVectorizer().fit(col)
    transformed = vect.transform(col)
    for i, feature_name in enumerate(vect.get_feature_names_out()):
        df[f'{column}_{feature_name}'] = transformed[:, i].toarray().flatten()  # Convert to dense array
    return df

for item in ['title', 'h1', 'abstract', 'url_text']:
    df = vectorize_text(column=item, df=df)
    filename = f'../data/full_data_{item}.csv'
    df.to_csv(filename, encoding='utf-8', index=False)


In [ ]:
df.to_csv('./data/full_data_vectorized.csv', encoding='utf-8', index=False)

In [ ]:
sorted(vect.vocabulary_.items(), key=lambda x: x[1])[:20]

In [ ]:
print("Text dataframe shape = {}".format(col.shape))
print("Vocabulary length = {}".format(len(vect.vocabulary_)))

Make pretty & remove some columns

In [ ]:
col_order = ['version_id_ong', 'page_id', 'publish_date', 'date', 'last_update',
             'url_text', 'page_name', 'title', 'h1', 'abstract', 
             'classification_product', 'classification_type', 
             'authors', 'author_scraped',
             'image_url', 'word_count', 'words_scraped', 
             'daily_likes', 'daily_dislikes', 'video_play', 'page_impressions', 'clickouts', 
             'external_clicks', 'external_impressions', 'ctr'
             ]
df_eng = df[col_order]